## TASK 1:
# Import Necessary Libraries
This cell installs and imports the necessary libraries for the tasks. These include:

requests for making API calls.
beautifulsoup4 for parsing HTML content during web scraping.
pandas for handling and manipulating datasets.
time for adding delays in requests to avoid overloading APIs or web servers.
tqdm for creating progress bars to monitor the progress of tasks.

In [1]:
# Import necessary libraries
!pip install requests beautifulsoup4
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

# Load the Movies Dataset
This cell loads the dataset containing information about movies. The dataset is expected to be in a CSV file named movies_set1.csv. The cell also displays the first few rows of the dataset using head() to ensure it has been loaded correctly.

In [ ]:
# Load the movies dataset
movies_path = 'movies_set1.csv'  # Adjust path accordingly
movies_df = pd.read_csv(movies_path)
movies_df.head()

,movieId,title,genres
0,120510,Value for Money (1955),Comedy|Romance
1,212955,Face of Evil (1996),Drama|Thriller
2,193912,Spring 1941 (2007),Drama|Romance|War
3,163921,Wolf Creek (2016),Crime|Horror|Thriller
4,126652,Raven the Little Rascal (2012),Animation|Children


# Set TMDb API Key
This cell defines the API key required for making requests to The Movie Database (TMDb) API.

In [ ]:
# TMDb API key
API_KEY = '4aad8158e12f512a80a29ee5540098dc'

# Define Base URLs for TMDb API

This cell sets up the base URLs for making requests to The Movie Database (TMDb) API. The search_url is used for searching movies by title, and credits_url_template is used to retrieve the cast (actors) for a given movie by its ID.

In [ ]:
# Base URLs for TMDb API
search_url = 'https://api.themoviedb.org/3/search/movie'
credits_url_template = 'https://api.themoviedb.org/3/movie/{movie_id}/credits'

In [ ]:

# Prepare a dictionary to store movie-actor relationships
movie_actor_dict = {}

# Fetching Top Actors for Movies Using TMDb API
In this step, we fetch the top 5 actors for each movie in the dataset by querying the TMDb API. This helps link movies to their actors, which is essential for creating the user-actor rating matrix later on.

The get_top_actors function searches for the movie by title, retrieves its ID, and then fetches the cast details. It extracts the top 5 actors from the cast and returns them. If any errors occur during this process, such as network issues, an empty list is returned for that movie.

To speed up the process, we use ThreadPoolExecutor to make parallel API requests. This allows us to fetch actor details for multiple movies simultaneously. The results are stored in a dictionary with movie titles as keys and actor lists as values. We also use tqdm to show the progress of the task, providing real-time feedback on the number of completed requests.

This approach optimizes the process of fetching actor data, making it faster and more efficient for large datasets.

In [ ]:
# from concurrent.futures import ThreadPoolExecutor, as_completed
# # Function to get movie ID and then retrieve top actors
# def get_top_actors(movie_title):
#     try:
#         # Search for the movie by title
#         search_params = {'api_key': API_KEY, 'query': movie_title}
#         search_response = requests.get(search_url, params=search_params).json()

#         if search_response['results']:
#             movie_id = search_response['results'][0]['id']  # Get the first matching movie ID

#             # Get the credits (cast) for the movie
#             credits_url = credits_url_template.format(movie_id=movie_id)
#             credits_params = {'api_key': API_KEY}
#             credits_response = requests.get(credits_url, params=credits_params).json()

#             # Extract the top 5 actors
#             actors = [cast['name'] for cast in credits_response.get('cast', [])[:5]]
#             return movie_title, actors
#         else:
#             return movie_title, []
#     except Exception as e:
#         print(f"Error fetching actors for {movie_title}: {e}")
#         return movie_title, []

# # Use ThreadPoolExecutor to parallelize API requests
# with ThreadPoolExecutor(max_workers=10) as executor:
#     future_to_movie = {executor.submit(get_top_actors, row['title']): row['title'] for _, row in movies_df.iterrows()}
#     for future in tqdm(as_completed(future_to_movie), total=len(future_to_movie)):
#         movie_title, actors = future.result()
#         movie_actor_dict[movie_title] = actors

import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
import time

# Load MovieLens movies (must contain movieId and title)
movies_df = pd.read_csv("movies_set1.csv")   # e.g., user-movie mappings

API_KEY = "YOUR_TMDB_API_KEY"   # Insert your TMDb key
search_url = "https://api.themoviedb.org/3/search/movie"
credits_url = "https://api.themoviedb.org/3/movie/{movie_id}/credits"

def get_top_actors(movie_title):
    """
    Search TMDb movie_id for the title, then fetch top 5 actors.
    """
    try:
        response = requests.get(search_url, params={"api_key": API_KEY, "query": movie_title})
        data = response.json()

        if not data["results"]:
            return None, None

        movie_id = data["results"][0]["id"]

        # Fetch credits
        credits_response = requests.get(
            credits_url.format(movie_id=movie_id),
            params={"api_key": API_KEY}
        ).json()

        cast = credits_response.get("cast", [])
        top_actors = [actor["name"] for actor in cast[:5]]

        return movie_id, top_actors

    except Exception:
        return None, None


movie_actor_list = []

def process_row(row):
    title = row["title"]
    movieId = row["movieId"]

    movie_id_db, actors_list = get_top_actors(title)

    # Skip missing
    if movie_id_db is None or actors_list is None:
        return

    movie_actor_list.append({
        "movieId": movieId,          # MovieLens ID
        "movie_title": title,
        "movie_id_db": movie_id_db,  # TMDb ID
        "actor_list": actors_list    # List of 5 actors
    })


# Run scraping in parallel
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(process_row, movies_df.to_dict("records"))

# Convert to DataFrame
movie_actors_df = pd.DataFrame(movie_actor_list)

# Save
movie_actors_df.to_csv("movie_actors.csv", index=False)
print("movie_actors.csv created successfully!")
movie_actors_df.head()


# Saving Movie and Actor Data to CSV
After retrieving the top 5 actors for each movie, the next step is to organize and save the results for further use. This cell creates a new DataFrame from the movie_actor_dict dictionary, where each movie title is paired with a list of top actors. The DataFrame is then exported as a CSV file (movie_actors.csv), making it easy to access and work with the movie-actor associations in future tasks.

In [ ]:
# # Save the results to a new DataFrame and export it as a CSV
# movie_actors_df = pd.DataFrame(list(movie_actor_dict.items()), columns=['Movie Title', 'Top Actors'])
# movie_actors_df.to_csv('movie_actors.csv', index=False)  # Adjust path accordingly

# print("Top actors data saved successfully.")


# STEP 2 — Convert actor list to one actor per row

# This will create movie_actor_relationships.csv having:

# | movieId | movie_title | movie_id_db | actor_name |

import pandas as pd

# Load the scraped file
df = pd.read_csv("movie_actors.csv")

# Convert string list → actual Python list
df["actor_list"] = df["actor_list"].apply(eval)

# Expand list into multiple rows
exploded_df = df.explode("actor_list")

# Rename for clarity
exploded_df = exploded_df.rename(columns={
    "actor_list": "actor_name"
})

# Save final relationships file
exploded_df.to_csv("movie_actor_relationships.csv", index=False)

print("movie_actor_relationships.csv created successfully!")
exploded_df.head()


Top actors data saved successfully.


# TASK 2
# Constructing the User-Actor Rating Matrix
This cell outlines the process of constructing the user-actor rating matrix from the user-movie ratings and movie-actor associations. The steps are as follows:

Loading Datasets: The user-movie ratings (ratings_set1.csv) and movie-actor associations (movie_actors.csv) are loaded into Pandas DataFrames.

Data Preparation: The movie IDs in both datasets are converted to strings to ensure they match properly during the merge process.

Exploding the Actor List: The top_actors column, which contains comma-separated actor names, is converted into a list. Then, the list is "exploded" so that each actor appears on a separate row for the corresponding movie.

Merging Data: The ratings data is merged with the expanded movie-actor data to associate each user's rating with the corresponding actors of the movies they have rated.

Calculating Average Ratings: For each user-actor pair, the average rating is computed, which represents how much a user likes an actor based on the movies they have rated.

Creating the User-Actor Rating Matrix: The long-format data is converted into a pivot table (user-actor rating matrix), which is useful for recommendation algorithms.

Saving Results: The user-actor ratings and matrix are saved as CSV files (user_actor_ratings.csv and user_actor_matrix.csv) for further use in model training and evaluation.

The resulting DataFrame provides the necessary structure for applying recommendation algorithms and generating personalized actor recommendations for users based on their preferences.

In [2]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357278 sha256=6b19d0e5afdf8d1a55cc75180e92b1abe8e82e0b7410c95953f31fd33b161a88
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd

# Load the datasets
ratings = pd.read_csv('/content/ratings_set1.csv')  # User-movie ratings
movie_actors = pd.read_csv('/content/movie_actor_relationships.csv')  # Movie-actor associations

# Print the columns to check for discrepancies in column names
print("Ratings Columns:", ratings.columns)
print("Movie Actors Columns:", movie_actors.columns)

# Strip any extra spaces in column names
ratings.columns = ratings.columns.str.strip()
movie_actors.columns = movie_actors.columns.str.strip()

# Check again for correct column names
print("Updated Ratings Columns:", ratings.columns)
print("Updated Movie Actors Columns:", movie_actors.columns)

# Ensure 'movieId' column is in the ratings dataset and 'movie_id_db' is in the movie_actors dataset
if 'movieId' not in ratings.columns:
    raise ValueError("Column 'movieId' not found in the ratings dataset")
if 'movie_id_db' not in movie_actors.columns:
    raise ValueError("Column 'movie_id_db' not found in the movie_actors dataset")

# Merge ratings with movie_actors on movieId (ratings dataset) and movie_id_db (movie_actors dataset)
user_movie_actors = pd.merge(ratings, movie_actors[['movie_id', 'actor_name']],
                             left_on='movieId', right_on='movie_id', how='inner')

# Group by userId and actor_name, and calculate the average rating per actor
user_actor_ratings = (
    user_movie_actors.groupby(['userId', 'actor_name'])['rating']
    .mean()
    .reset_index()
    .rename(columns={'rating': 'average_rating'})
)

# Optional: Preprocessing to remove low-activity users and actors
# Filter out actors with very few ratings
min_ratings_per_actor = 5
actor_rating_counts = user_actor_ratings['actor_name'].value_counts()
active_actors = actor_rating_counts[actor_rating_counts >= min_ratings_per_actor].index
user_actor_ratings = user_actor_ratings[user_actor_ratings['actor_name'].isin(active_actors)]

# Filter out users with very few ratings
min_ratings_per_user = 5
user_rating_counts = user_actor_ratings['userId'].value_counts()
active_users = user_rating_counts[user_rating_counts >= min_ratings_per_user].index
user_actor_ratings = user_actor_ratings[user_actor_ratings['userId'].isin(active_users)]

# Save the user-actor rating dataset to a CSV file
user_actor_ratings.to_csv('/content/user_actor_ratings.csv', index=False)

# Print a success message and display the first few rows of the saved dataset
print("User-Actor rating dataset has been saved as 'user_actor_ratings.csv'.")
print(user_actor_ratings.head())


Ratings Columns: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
Movie Actors Columns: Index(['movie_id_db', 'movie_title', 'actor_name'], dtype='object')
Updated Ratings Columns: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
Updated Movie Actors Columns: Index(['movie_id_db', 'movie_title', 'actor_name'], dtype='object')
User-Actor rating dataset has been saved as 'user_actor_ratings.csv'.
   userId           actor_name  average_rating
0       5        Aaron Izbicki             4.0
1       5  Aaron Michael Lacey             4.0
2       5        Afemo Omilami             4.0
3       5  Afram Bill Williams             4.0
4       5        Al Harrington             4.0


In [ ]:
print(movie_actors.columns)

Index(['movie_id_db', 'movie_title', 'actor_name'], dtype='object')


# TASK 3
# Model Training and Evaluation with SVD
This cell implements the training and evaluation of the Singular Value Decomposition (SVD) algorithm using the Surprise library. The steps are outlined as follows:

Loading the Data: The user-actor rating data (user_actor_ratings.csv) is loaded into a Pandas DataFrame. This data contains ratings of actors by different users, and it is essential for building the recommendation model.

Setting Up the Surprise Reader: A Reader object is defined with the rating scale. The minimum and maximum values of the avg_actor_rating column are extracted from the data and passed to the Reader. This defines the range of the ratings for the Surprise algorithm.

Loading the Data into Surprise Dataset: The Dataset.load_from_df() method is used to convert the user-actor rating DataFrame into a Surprise dataset format. This dataset contains three columns: userId, actorId, and avg_actor_rating.

Initializing the SVD Algorithm: The Singular Value Decomposition (SVD) algorithm is initialized using SVD(). SVD is a popular matrix factorization technique used in collaborative filtering for making recommendations.

Cross-Validation: The cross_validate() function is used to perform 5-fold cross-validation on the SVD algorithm. This function splits the data into five parts, trains the model on four parts, and evaluates it on the fifth. It calculates two common evaluation metrics: RMSE (Root Mean Squared Error) and MAE (Mean Absolute Error).

Results: The evaluation results (RMSE and MAE) are printed out, providing insights into the performance of the model on the user-actor rating data.

By performing cross-validation, we can assess the generalization ability of the model and its performance in predicting actor preferences for users.

In [22]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

# Load the user-actor rating data
average_rating = pd.read_csv('/content/user_actor_ratings.csv')  # from Task 2

# Define a reader for Surprise with the expected rating scale (1-5 for MovieLens)
reader = Reader(rating_scale=(average_rating['average_rating'].min(),
                              average_rating['average_rating'].max()))

# Load the data into Surprise Dataset
data = Dataset.load_from_df(average_rating[['userId', 'actor_name', 'average_rating']], reader)

# Initialize the SVD algorithm
svd_algo = SVD()

# Train and evaluate the model using cross-validation (e.g., 3-fold)
cross_val_results = cross_validate(svd_algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

# Output results
print("Cross-Validation Results (RMSE, MAE):")
print(cross_val_results)


Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.6677  0.6698  0.6688  0.6688  0.0009  
MAE (testset)     0.4669  0.4682  0.4671  0.4674  0.0006  
Fit time          54.96   57.55   56.61   56.37   1.07    
Test time         19.09   21.83   19.05   19.99   1.30    
Cross-Validation Results (RMSE, MAE):
{'test_rmse': array([0.66768767, 0.66984269, 0.66878309]), 'test_mae': array([0.46694471, 0.4681875 , 0.46707722]), 'fit_time': (54.96441674232483, 57.548688650131226, 56.610989809036255), 'test_time': (19.094894886016846, 21.830989599227905, 19.05217742919922)}


# TASK 4
# Actor Recommendation for Users Using SVD
This cell focuses on using the trained Singular Value Decomposition (SVD) model to generate personalized actor recommendations for users. Here's how the code works:

Loading Data and Training the Model:

The user_actor_ratings.csv dataset is loaded, which contains user ratings for different actors. The dataset is transformed into a format suitable for the Surprise library using the Reader and Dataset classes.
The full training set is created using build_full_trainset() and the SVD model is trained with svd_algo.fit().
Generating Actor Recommendations:

A list of all unique actor IDs is extracted from the user_actor_ratings DataFrame. This will be used to generate predictions for all actors that a given user has not yet rated.
The recommend_actors function is designed to recommend top-N actors for a specific user. It takes into account the following:
Already Rated Actors: The function filters out the actors the user has already rated, ensuring predictions are only made for unrated actors.
Predicted Ratings: For each unrated actor, the model predicts a rating using model.predict(user_id, actor_id).est. These predicted ratings are stored and sorted in descending order to prioritize actors the model believes the user would like most.
Example Usage:

The function is called with user_id = 1 to generate the top 10 actor recommendations. The predicted ratings for these actors are displayed in descending order.
Output:

The top-10 recommended actors for the given user (ID = 1) are printed along with their predicted ratings, which represent how much the user is likely to enjoy the actor's work.
By using this approach, you can provide personalized recommendations of actors based on users' previous ratings, facilitating a tailored movie experience for each individual.

In [5]:
import pandas as pd

# Load the user-actor rating data
average_rating = pd.read_csv('/content/user_actor_ratings.csv')  # from Task 2

# Function to generate top-N actor recommendations for a given user using the trained model
def recommend_actors(user_id, model, dataset, n=10):
    # Get all unique actors
    all_actors = dataset['actor_name'].unique()

    # Get the list of actors the user has already rated
    user_rated_actors = dataset[dataset['userId'] == user_id]['actor_name'].tolist()

    # Filter out actors the user has already rated
    actors_to_predict = [actor for actor in all_actors if actor not in user_rated_actors]

    # Predict the rating for each actor and store in a list
    predictions = []
    for actor in actors_to_predict:
        pred = model.predict(user_id, actor)
        predictions.append((actor, pred.est))

    # Sort the predictions by estimated rating in descending order
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Return the top-N recommendations
    return predictions[:n]

# Example usage: Generate recommendations for a specific user
user_id_to_recommend = 1  # Replace with a valid user ID
top_n_recommendations = recommend_actors(user_id_to_recommend, svd_algo, average_rating, n=10)

# Display the top-N recommendations
print(f"Top {len(top_n_recommendations)} actor recommendations for user {user_id_to_recommend}:")
for actor, rating in top_n_recommendations:
    print(f"{actor}: Predicted Rating = {rating:.2f}")


Top 10 actor recommendations for user 1:
Mario Brega: Predicted Rating = 4.52
Eddie Parker: Predicted Rating = 4.49
Teresa Harder: Predicted Rating = 4.49
Jimmy Grant: Predicted Rating = 4.48
Morgan Lund: Predicted Rating = 4.46
Raymond Burr: Predicted Rating = 4.45
Joseph Ragno: Predicted Rating = 4.44
Gunnel Lindblom: Predicted Rating = 4.44
Brian Delate: Predicted Rating = 4.44
Carol Reed: Predicted Rating = 4.44


# TASK 5
# Evaluation of Actor Recommendation System using SVD
This cell evaluates a recommendation system that suggests top-N actors to a user based on their historical ratings using Singular Value Decomposition (SVD). The evaluation metrics used are Precision@k, Recall@k, and NDCG@k, which are standard measures for evaluating the quality of recommendations in collaborative filtering systems.

Details of the Code:
Data Loading:

The user-actor ratings dataset is loaded, which contains columns: userId, actorId, and avg_actor_rating. These represent ratings given by users to actors.
SVD Model Training:

An SVD model from the surprise library is used to model user-actor interactions. This model is trained on the complete user-actor ratings dataset using the Reader class from the surprise library to handle the rating scale.
Actor Recommendations:

The function recommend_actors takes a user_id and uses the trained SVD model to predict ratings for actors that the user has not yet rated. It then returns the top-N recommended actors based on the predicted ratings.
Evaluation Metrics:

Precision@k: Measures the proportion of recommended actors that are relevant (i.e., the user has already rated them).
Recall@k: Measures the proportion of relevant actors that are recommended among the top-N recommendations.
NDCG@k: Measures the ranking quality of the top-N recommendations by considering the position of relevant actors (discounted cumulative gain).
Evaluation Process:

The function evaluate_recommendations calculates Precision@k, Recall@k, and NDCG@k for each user in the dataset.
It loops through each user, generates the top-N recommendations using recommend_actors, and then calculates the three evaluation metrics.
The final evaluation results are the average scores across all users.
Results Display:

After evaluating the recommendation system, the precision, recall, and NDCG values at k=10 are printed to provide insights into the quality of the recommendations.

In [19]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import train_test_split

# Load the user-actor rating data (from Task 2)
user_actor_ratings = pd.read_csv('/content/user_actor_ratings.csv')

# Define a reader for Surprise with the expected rating scale
reader = Reader(rating_scale=(user_actor_ratings['average_rating'].min(), user_actor_ratings['average_rating'].max()))

# Load the data into Surprise's format
data = Dataset.load_from_df(user_actor_ratings[['userId', 'actor_name', 'average_rating']], reader)

# Load the train and test sets from the pre-split data
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Assuming you have a pre-trained model (svd_algo) from previous steps
# If not, use: svd_algo.fit(trainset) to train it
# Generate predictions for the test set using the pre-trained model
predictions = svd_algo.test(testset)


In [20]:

# Specify the user ID for evaluation
specific_user_id = 5  # Replace with the desired user ID

# Filter predictions for the specific user
user_predictions = [pred for pred in predictions if pred.uid == specific_user_id]

# Sort predictions by estimated rating in descending order
user_predictions.sort(key=lambda x: x.est, reverse=True)

# Function to calculate Precision@k
def precision_at_k(predictions, k):
    relevant_retrieved = sum([1 for _, _, true_r, est, _ in predictions[:k] if true_r >= 3.5 and est >= 3.5])
    return relevant_retrieved / k if k > 0 else 0

# Function to calculate Recall@k
def recall_at_k(predictions, k):
    relevant_retrieved = sum([1 for _, _, true_r, est, _ in predictions[:k] if true_r >= 3.5 and est >= 3.5])
    relevant_items = sum([1 for _, _, true_r, _, _ in predictions if true_r >= 3.5])
    return relevant_retrieved / relevant_items if relevant_items > 0 else 0

# Function to calculate NDCG@k (Normalized Discounted Cumulative Gain)
def ndcg_at_k(predictions, k):
    # Calculate DCG (Discounted Cumulative Gain)
    dcg = 0
    for i, (_, _, true_r, est, _) in enumerate(predictions[:k]):
        if true_r >= 3.5:  # Only consider relevant items (rating >= 3.5)
            dcg += (2 ** true_r - 1) / np.log2(i + 2)  # i + 2 because log2 starts from 2

    # Calculate IDCG (Ideal DCG) - best possible ranking
    relevant_predictions = sorted([true_r for _, _, true_r, _, _ in predictions if true_r >= 3.5], reverse=True)
    idcg = 0
    for i, true_r in enumerate(relevant_predictions[:k]):
        idcg += (2 ** true_r - 1) / np.log2(i + 2)

    # Return NDCG (normalized DCG)
    return dcg / idcg if idcg > 0 else 0

# Define k for evaluation
k = 10

# Evaluate metrics for the specific user
precision_k = precision_at_k(user_predictions, k)
recall_k = recall_at_k(user_predictions, k)
ndcg_k = ndcg_at_k(user_predictions, k)

# Output the results
print(f"User ID: {specific_user_id}")
print(f"Precision@{k}: {precision_k:.4f}")
print(f"Recall@{k}: {recall_k:.4f}")
print(f"NDCG@{k}: {ndcg_k:.4f}")


User ID: 5
Precision@10: 0.9000
Recall@10: 0.3214
NDCG@10: 0.9306
